## Решение практической задачи 1

### Задача: ранжирование потенциальных кандидатов по профессиональным обязанностям

> Решение практической задачи выполняется в два этапа. На первом этапе необходимо использовать библиотеку OCEAN-AI для получения гипотез предсказаний (оценок персональных качеств личности человека). На втором этапе следует использовать методы _candidate_ranking и _priority_skill_calculation из библиотеки OCEAN-AI для решения представленной практической задачи. Примеры результатов работы и реализации представлены ниже.

> Таким образом, библиотека OCEAN-AI предоставляет инструменты для анализа персональных качеств личности кандидатов и их соответствия требованиям должности, что может значительно улучшить процесс подбора персонала и помочь в принятии более объективных и систематизированных решений при ранжировании кандидатов.

<center>
    <img src="https://raw.githubusercontent.com/aimclub/OCEANAI/main/docs/source/user_guide/notebooks/images/Step_1_get_scores_en.svg" width="100%"/>
</center>

<center>
    <img src="https://raw.githubusercontent.com/aimclub/OCEANAI/main/docs/source/user_guide/notebooks/images/Step_2_task_1_en.svg" width="100%"/>
</center>

<hr>

In [9]:
# pip install -r docs/requirements.txt

In [10]:
import os  # Взаимодействие с файловой системой
import sys # Доступ к некоторым переменным и функциям Python

PATH_TO_SOURCE = os.path.abspath(os.path.dirname(globals()['_dh'][0]))
PATH_TO_ROOT = os.path.join(PATH_TO_SOURCE, '..', '..', '..')

sys.path.insert(0, os.path.abspath(PATH_TO_ROOT))

### `FI V2`

In [11]:
# Импорт необходимых инструментов
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

corpus = 'fi'

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url, force_reload = False)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang='en')
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['fe']['googledisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url, force_reload = False)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_bert_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['googledisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['text'][corpus]['nn']['googledisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url, force_reload = False)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['googledisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url, force_reload = False)

PATH_TO_DIR = './video_FI/'
PATH_SAVE_VIDEO = './video_FI/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '429713680?token=FqHdMLSSh7zYSZt&filename=_plk5k7PBEg.003.mp4',
    '429713681?token=Hz9b4lQkrLfic33&filename=be0DQawtVkE.002.mp4',
    '429713683?token=EgUXS9Xs8xHm5gz&filename=2d6btbaNdfo.000.mp4',
    '429713684?token=1U26753kmPYdIgt&filename=300gK3CnzW0.003.mp4',
    '429713685?token=LyigAWLTzDNwKJO&filename=300gK3CnzW0.001.mp4',
    '429713686?token=EpfRbCKHyuc4HPu&filename=cLaZxEf1nE4.004.mp4',
    '429713687?token=FNTkwqBr4jOS95l&filename=g24JGYuT74A.004.mp4',
    '429713688?token=qDT95nz7hfm2Nki&filename=JZNMxa3OKHY.000.mp4',
    '429713689?token=noLguEGXDpbcKhg&filename=nvlqJbHk_Lc.003.mp4',
    '429713679?token=9L7RQ0hgdJlcek6&filename=4vdJGgZpj4k.003.mp4'
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mp4'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_[corpus]['googledisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = 'en')

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:05:02</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:05:02</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... kaggle/working/OCEANAI/video_FI/test/4vdJGgZpj4k.003.mp4 ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
Person ID,,,,,,
1,be0DQawtVkE.002.mp4,0.676612,0.568144,0.552115,0.646172,0.640899
2,cLaZxEf1nE4.004.mp4,0.663619,0.5505,0.556719,0.585134,0.587168
3,g24JGYuT74A.004.mp4,0.586715,0.395457,0.407597,0.528028,0.502627
4,nvlqJbHk_Lc.003.mp4,0.511955,0.463753,0.391108,0.443741,0.438813
5,JZNMxa3OKHY.000.mp4,0.604756,0.522703,0.528715,0.593952,0.578697
6,_plk5k7PBEg.003.mp4,0.646188,0.609137,0.521393,0.612698,0.604051
7,2d6btbaNdfo.000.mp4,0.623021,0.661305,0.477141,0.655363,0.60524
8,300gK3CnzW0.003.mp4,0.468087,0.448507,0.372154,0.509777,0.453799
9,300gK3CnzW0.001.mp4,0.460228,0.411986,0.414164,0.497072,0.429769


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:05:02</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.073,0.0633,0.091,0.0703,0.0692,0.0734
Accuracy,0.927,0.9367,0.909,0.9297,0.9308,0.9266


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:05:02</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0734</span>, средняя точность: <span style="color:#1776D2">0.9266</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 88.396 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [12]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [13]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_fi_en', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
1,be0DQawtVkE.002.mp4,0.677,0.568,0.552,0.646,0.641,64.419
7,2d6btbaNdfo.000.mp4,0.623,0.661,0.477,0.655,0.605,62.365
6,_plk5k7PBEg.003.mp4,0.646,0.609,0.521,0.613,0.604,62.305
2,cLaZxEf1nE4.004.mp4,0.664,0.551,0.557,0.585,0.587,61.807
10,4vdJGgZpj4k.003.mp4,0.584,0.616,0.495,0.604,0.586,58.757
5,JZNMxa3OKHY.000.mp4,0.605,0.523,0.529,0.594,0.579,58.312
3,g24JGYuT74A.004.mp4,0.587,0.395,0.408,0.528,0.503,52.765
4,nvlqJbHk_Lc.003.mp4,0.512,0.464,0.391,0.444,0.439,47.748
8,300gK3CnzW0.003.mp4,0.468,0.449,0.372,0.510,0.454,46.446


#### Ранжирование кандидатов на должность менеджера

In [14]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_fi_en', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
7,2d6btbaNdfo.000.mp4,0.623,0.661,0.477,0.655,0.605,62.335
1,be0DQawtVkE.002.mp4,0.677,0.568,0.552,0.646,0.641,60.906
6,_plk5k7PBEg.003.mp4,0.646,0.609,0.521,0.613,0.604,60.235
10,4vdJGgZpj4k.003.mp4,0.584,0.616,0.495,0.604,0.586,58.792
2,cLaZxEf1nE4.004.mp4,0.664,0.551,0.557,0.585,0.587,58.062
5,JZNMxa3OKHY.000.mp4,0.605,0.523,0.529,0.594,0.579,56.009
3,g24JGYuT74A.004.mp4,0.587,0.395,0.408,0.528,0.503,47.110
8,300gK3CnzW0.003.mp4,0.468,0.449,0.372,0.510,0.454,45.864
4,nvlqJbHk_Lc.003.mp4,0.512,0.464,0.391,0.444,0.439,45.284


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [15]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [16]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_fi_en', out = True)

# Опционно
df = _b5.df_files_priority_skill_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Analytical,Interactive,Routine,Non-Routine
Person ID,,,,,,,,,,
7,2d6btbaNdfo.000.mp4,0.623,0.661,0.477,0.655,0.605,0.398,0.453,0.572,0.493
1,be0DQawtVkE.002.mp4,0.677,0.568,0.552,0.646,0.641,0.384,0.416,0.570,0.461
6,_plk5k7PBEg.003.mp4,0.646,0.609,0.521,0.613,0.604,0.378,0.413,0.562,0.455
10,4vdJGgZpj4k.003.mp4,0.584,0.616,0.495,0.604,0.586,0.374,0.426,0.537,0.463
2,cLaZxEf1nE4.004.mp4,0.664,0.551,0.557,0.585,0.587,0.359,0.390,0.537,0.433
5,JZNMxa3OKHY.000.mp4,0.605,0.523,0.529,0.594,0.579,0.352,0.380,0.522,0.421
3,g24JGYuT74A.004.mp4,0.587,0.395,0.408,0.528,0.503,0.238,0.247,0.329,0.296
8,300gK3CnzW0.003.mp4,0.468,0.449,0.372,0.510,0.454,0.199,0.154,0.198,0.202
9,300gK3CnzW0.001.mp4,0.460,0.412,0.414,0.497,0.430,0.093,0.072,0.073,0.104


<hr>

Для ранжирования кандидатов по одному из шестнадцати типов личности MBTI необходимо задать матрицу корреляции между персональными качествами личности человека и четырьмя диспозициями MBTI, установить порог полярности качеств и указать целевой тип личности MBTI.

В качестве примера предлагается использование коэффициентов корреляции, представленных в статье [1]. Описание типов личности MBTI и соотвествующие им успешные профессии представлены в статье [2].

1) Furnham A. The big five facets and the MBTI: The relationship between the 30 NEO-PI (R) Facets and the four Myers-Briggs Type Indicator (MBTI) scores // Psychology. - 2022. vol. 13(10). - pp. 1504-1516.
2) Tieger P.D., Barron B., Tieger K. Do what you are: Discover the perfect career for you through the secrets of personality type // Hachette UK. - 2024.

##### Типы личности MBTI основаны на четырех измерениях личности:

<div style="text-align: center;">
  <table style="width:100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Описание измерения</th>
      <th>Измерение</th>
    </tr>
    <tr>
      <td>Как мы взаимодействуем с миром и куда направляем свою энергию</td>
      <td>(E) Экстраверсия - Интроверсия (I)</td>
    </tr>
    <tr>
      <td>Вид информации, которую мы естественным образом замечаем</td>
      <td>(S) Сенсорика - Интуиция (N)</td>
    </tr>
    <tr>
      <td>Как мы принимаем решения</td>
      <td>(T) Логика - Чувства (F)</td>
    </tr>
    <tr>
      <td>Предпочитаем ли мы жить более структурированно (принимая решения) или более спонтанно (принимая информацию)</td>
      <td>(J) Оценка - Восприятие (P)</td>
    </tr>
  </table>
</div>

##### Типы личности MBTI и соотвествующие им успешные профессии:

<div style="text-align: center;">
  <table style="width: 100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Тип личности</th>
      <th>Описание</th>
      <th>Успешные профессии</th>
    </tr>
    <tr>
      <td>ISTJ</td>
      <td>Этот человек отличается ответственностью, строгостью и педантичностью. Он опирается на объективные факты и склонен к аналитическому мышлению. Приступает к задаче только тогда, когда уверен в своих возможностях и успехе</td>
      <td>Инспектор: бухгалтер, аудитор, бюджетный аналитик, финансовый менеджер, разработчик, системный аналитик, библиотекарь и т. д.</td>
    </tr>
    <tr>
      <td>ISFJ</td>
      <td>Этот человек склонен к самоанализу и анализу окружающих, легко распознает фальшь и предпочитает сохранять психологическую дистанцию. Он исполнителен, внимателен и готов помогать другим. Его силы и энергия исходят из внутренних ресурсов, и он всегда полагается на собственный опыт</td>
      <td>Защитник: медсестра, врач, ветеринар или ветеринарный ассистент, социальный работник, сельскохозяйственный или пищевой ученый, секретарь, водитель и т. д.</td>
    </tr>
    <tr>
      <td>INFJ</td>
      <td>О таких людях говорят: «ему можно доверять». Он отличается высокой чувствительностью, уделяет большое внимание межличностным отношениям, умеет давать ценные советы и помогает раскрывать потенциал других. Развитая интуиция не только генерирует множество идей, но и способствует самоорганизации</td>
      <td>Советник: психолог, специалист по управлению персоналом, офис-менеджер, специалист по обучению, графический дизайнер и т. д.</td>
    </tr>
    <tr>
      <td>INTJ</td>
      <td>Этот человек умеет выделять главное, говорит четко и по существу, придерживается практического подхода. Он стремится постоянно улучшать свою работу и всегда ищет способы сделать задачу еще лучше. Пустые разговоры ему не по душе, поэтому он избегает больших шумных компаний и с трудом заводит новые знакомства</td>
      <td>Мастермайнд: аниматор, архитектор, копирайтер, фотограф, тележурналист, видеомонтажер, специалист по бизнес-развитию, исполнительный директор, профессор и т. д.</td>
    </tr>
    <tr>
      <td>ISTP</td>
      <td>Этот человек воспринимает мир через ощущения. По природе эмпат, но чаще сосредоточен на себе. Его умение объективно принимать решения и анализировать ситуацию указывает на технический склад ума. Он всегда соблюдает дедлайны, хотя иногда может поступить неожиданно</td>
      <td>Создатель: инженер, техник, строитель, инспектор, судебный эксперт, программист, разработчик ПО и т. д.</td>
    </tr>
    <tr>
      <td>ISFP</td>
      <td>Этот человек умеет находить радость в однообразии и рутинных делах. Прекрасно ладит с людьми, избегая конфликтов. Ему важно чувствовать свою значимость и оказывать помощь. Такой человек не стремится руководить или менять других, уважает их личные границы и ожидает того же в ответ. По натуре он приземленный практик, на которого всегда можно положиться</td>
      <td>Композитор: помощник по маркетингу, танцор, шеф-повар, офис-администратор, художник, дизайнер интерьеров, секретарь, медсестра и т. д.</td>
    </tr>
    <tr>
      <td>INFP</td>
      <td>Этот человек - чувствительный лирик, прекрасно разбирающийся в людях и легко вызывающий у них симпатию. Он обладает отличным чувством юмора и уделяет большое внимание своему внешнему виду. Стремится к самопознанию, гармонии с собой и старается быть полезным окружающим</td>
      <td>Целитель: писатель, дизайнер мультимедиа, менеджер по работе с клиентами, учитель для детей с особыми потребностями, тренер, редактор, модельер и т. д.</td>
    </tr>
    <tr>
      <td>INTP</td>
      <td>Этот человек - эрудит с философским складом ума. Он тщательно анализирует свои решения, стремясь к объективности и беспристрастности. Бурные проявления эмоций ему не свойственны. Однако большое количество данных и их изменчивость могут вызывать у него внутреннее напряжение</td>
      <td>Архитектор: технический писатель, веб-разработчик, аналитик информационной безопасности, исследователь, ученый, юрист и т. д.</td>
    </tr>
    <tr>
      <td>ESTP</td>
      <td>Этот человек всегда добивается успеха, невзирая на препятствия, которые лишь усиливают его целеустремленность. Он стремится к лидерским позициям и плохо переносит роль подчиненного. Обычно разрабатывает четкий план действий и неуклонно ему следует</td>
      <td>Промоутер: специалист по работе с клиентами, актер, личный тренер, бренд-амбассадор, менеджер, предприниматель, креативный директор, полицейский, маркетолог, производитель и т. д.</td>
    </tr>
    <tr>
      <td>ESFP</td>
      <td>Этот человек легко выявляет слабые стороны людей, что позволяет ему эффективно манипулировать и управлять. В общении он чаще всего руководствуется собственными интересами и предпочитает жить в настоящем. Часто не завершает начатое, стремясь к быстрым результатам. Однако при этом стремится поддерживать гармоничные отношения с окружающими</td>
      <td>Исполнитель: бортпроводник, артист, учитель, менеджер по связям с общественностью, торговый представитель, организатор мероприятий и т. д.</td>
    </tr>
    <tr>
      <td>ENFP</td>
      <td>Этот человек - творческая личность и фантазер, обладающий качествами, которые помогают ему успешно взаимодействовать с другими, быть открытым и общительным. Он активно участвует в различных мероприятиях, легко решает возникающие вопросы и демонстрирует гибкость</td>
      <td>Чемпион: медицинский работник, продюсер, продавец-консультант, специалист по обслуживанию клиентов, сценарист, ведущий на ТВ/радио и т. д.</td>
    </tr>
    <tr>
      <td>ENTP</td>
      <td>Этот человек - изобретательный, инициативный и гибкий. Он генератор идей и первопроходец, который не выносит рутины. Постоянное движение и интуитивное принятие решений всегда сопровождают его в работе</td>
      <td>Новатор: инженер, маркетолог, менеджер по социальным сетям, аналитик управления, руководитель цифрового маркетинга, бизнес-консультант, разработчик игр, менеджер по продажам и т. д.</td>
    </tr>
    <tr>
      <td>ESTJ</td>
      <td>Это трудолюбивый человек, который воспринимает мир таким, какой он есть. Он склонен тщательно планировать и доводить дела до конца. Заботится о своем ближайшем окружении, проявляет добродушие, но иногда может быть вспыльчивым, резким и упрямым</td>
      <td>Супервайзер: управляющий директор, менеджер отеля, финансовый сотрудник, судья, агент по недвижимости, генеральный директор, шеф-повар, менеджер по бизнес-развитию, телемаркетолог и т. д.</td>
    </tr>
    <tr>
      <td>ESFJ</td>
      <td>Этот человек умеет оказывать влияние на людей, проявляет заботу и готов жертвовать собой ради других. Он легко устанавливает контакт с любым человеком и способен направить ситуацию в нужное ему русло</td>
      <td>Поставщик: специалист по технической поддержке, менеджер по работе с клиентами, профессор колледжа, медицинский исследователь, бухгалтер, фотожурналист и т. д.</td>
    </tr>
    <tr>
      <td>ENFJ</td>
      <td>Этот человек отличается эмоциональностью и эмпатией. Его мимика выразительна, а речь — красноречива. Благодаря своей самоорганизованности, он успешно воплощает свои фантазии и идеи в жизнь. Он интуитивно понимает, какое решение следует принять в каждой конкретной ситуации</td>
      <td>Учитель: менеджер по связям с общественностью, менеджер по продажам, директор по управлению персоналом, арт-директор, консультант и т. д.</td>
    </tr>
    <tr>
      <td>ENTJ</td>
      <td>Этот человек легко увлекается, готов рисковать и полагается на интуицию. Без страха внедряет новые технологии и способен глубоко анализировать как себя, так и окружающий мир. Жизнь для него - это борьба, в которой он чувствует себя уверенно. Открыт для новых возможностей, но при этом нуждается в контроле</td>
      <td>Командир: руководитель строительства, администратор службы здравоохранения, финансовый бухгалтер, аудитор, юрист, директор школы, химический инженер, менеджер баз данных и т. д.</td>
    </tr>
  </table>
</div>

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим типам личности.

#### Ранжирование кандидатов по одному из шестнадцати типов личности по версии MBTI

In [17]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/493644095?token=EX7hFxNJhMoLumI&filename=df_mbti_correlation.csv'
df_correlation_coefficients = pd.read_csv(url)

df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,EI,SN,TF,JP
ID,,,,,
1,Openness,0.09,-0.03,-0.14,-0.16
2,Conscientiousness,0.04,-0.04,0.20,0.14
3,Extraversion,0.20,-0.03,0.01,-0.07
4,Agreeableness,0.02,0.05,-0.35,0.03
5,Non-Neuroticism,0.08,0.00,0.16,0.00


In [18]:
_b5._professional_match(
    correlation_coefficients = df_correlation_coefficients,
    personality_type = "ENFJ",
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5._df_files_MBTI_job_match, name = 'MBTI_ranking_fi_en', out = True)

# Опционно
df = _b5.df_files_MBTI_job_match_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:6]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,EI,SN,TF,JP,MBTI,MBTI_Score,Match
Person ID,,,,,,,,,,,,,
1,be0DQawtVkE.002.mp4,0.677,0.568,0.552,0.646,0.641,0.258239,-0.027279,-0.099192,-0.047981,ENFP,0.288533,75.0
2,cLaZxEf1nE4.004.mp4,0.664,0.551,0.557,0.585,0.587,0.251766,-0.029373,-0.088090,-0.050525,ENFP,0.276922,75.0
6,_plk5k7PBEg.003.mp4,0.646,0.609,0.521,0.613,0.604,0.247379,-0.028758,-0.081221,-0.036228,ENFP,0.268019,75.0
5,JZNMxa3OKHY.000.mp4,0.605,0.523,0.529,0.594,0.579,0.239254,-0.025215,-0.090130,-0.042774,ENFP,0.265949,75.0
8,300gK3CnzW0.003.mp4,0.468,0.449,0.372,0.510,0.454,-0.160607,0.068636,-0.278920,0.053447,ISFJ,0.166184,50.0
7,2d6btbaNdfo.000.mp4,0.623,0.661,0.477,0.655,0.605,0.048622,0.001940,-0.092272,0.045960,ESFJ,0.140141,75.0
3,g24JGYuT74A.004.mp4,0.587,0.395,0.408,0.528,0.503,0.006237,0.036846,-0.269697,-0.104866,ESFP,0.137967,50.0
10,4vdJGgZpj4k.003.mp4,0.584,0.616,0.495,0.604,0.586,0.037174,0.002915,-0.081327,0.045557,ESFJ,0.123044,75.0
4,nvlqJbHk_Lc.003.mp4,0.512,0.464,0.391,0.444,0.439,-0.094676,-0.007262,-0.083236,-0.132773,INFP,0.045249,50.0


### `MuPTA` (ru)

In [19]:
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

corpus = 'mupta'
lang = 'ru'

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url, force_reload = False)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang=lang)
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['fe']['googledisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url, force_reload = False)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
res_setup_translation_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['googledisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['text'][corpus]['nn']['googledisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url, force_reload = False)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['googledisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url, force_reload = False)

PATH_TO_DIR = './video_MuPTA/'
PATH_SAVE_VIDEO = './video_MuPTA/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса MuPTA
# URL: https://hci.nw.ru/en/pages/mupta-corpus
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '477995979?token=2cvyk7CS0mHx2MJ&filename=speaker_06_center_83.mov',
    '477995980?token=jGPtBPS69uzFU6Y&filename=speaker_01_center_83.mov',
    '477995967?token=zCaRbNB6ht5wMPq&filename=speaker_11_center_83.mov',
    '477995966?token=B1rbinDYRQKrI3T&filename=speaker_15_center_83.mov',
    '477995978?token=dEpVDtZg1EQiEQ9&filename=speaker_07_center_83.mov',
    '477995961?token=o1hVjw8G45q9L9Z&filename=speaker_19_center_83.mov',
    '477995964?token=5K220Aqf673VHPq&filename=speaker_23_center_83.mov',
    '477995965?token=v1LVD2KT1cU7Lpb&filename=speaker_24_center_83.mov',
    '477995962?token=tmaSGyyWLA6XCy9&filename=speaker_27_center_83.mov',
    '477995963?token=bTpo96qNDPcwGqb&filename=speaker_10_center_83.mov',
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mov'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['mupta']['googledisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = lang)

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:15:48</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:15:52</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... kaggle/working/OCEANAI/video_MuPTA/test/speaker_11_center_83.mov ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
Person ID,,,,,,
1,speaker_01_center_83.mov,0.765416,0.696566,0.655036,0.758601,0.493137
2,speaker_15_center_83.mov,0.670995,0.670984,0.602225,0.708286,0.399343
3,speaker_07_center_83.mov,0.672468,0.662605,0.573035,0.705879,0.382358
4,speaker_19_center_83.mov,0.767356,0.658374,0.65355,0.801389,0.463584
5,speaker_27_center_83.mov,0.760904,0.710591,0.653713,0.827697,0.506974
6,speaker_10_center_83.mov,0.698376,0.599233,0.571993,0.674869,0.351017
7,speaker_24_center_83.mov,0.710313,0.662193,0.610217,0.71095,0.413197
8,speaker_06_center_83.mov,0.686585,0.659252,0.611847,0.749765,0.420683
9,speaker_23_center_83.mov,0.701222,0.688211,0.619921,0.807958,0.451005


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:15:52</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.0709,0.079,0.1324,0.1069,0.1,0.0978
Accuracy,0.9291,0.921,0.8676,0.8931,0.9,0.9022


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:15:52</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0978</span>, средняя точность: <span style="color:#1776D2">0.9022</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 567.515 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [20]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [21]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_mupta_ru', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
5,speaker_27_center_83.mov,0.761,0.711,0.654,0.828,0.507,71.993
1,speaker_01_center_83.mov,0.765,0.697,0.655,0.759,0.493,70.771
4,speaker_19_center_83.mov,0.767,0.658,0.654,0.801,0.464,70.486
9,speaker_23_center_83.mov,0.701,0.688,0.620,0.808,0.451,67.368
7,speaker_24_center_83.mov,0.710,0.662,0.610,0.711,0.413,65.362
8,speaker_06_center_83.mov,0.687,0.659,0.612,0.750,0.421,64.834
10,speaker_11_center_83.mov,0.718,0.599,0.573,0.732,0.380,64.460
2,speaker_15_center_83.mov,0.671,0.671,0.602,0.708,0.399,63.240
3,speaker_07_center_83.mov,0.672,0.663,0.573,0.706,0.382,62.751


#### Ранжирование кандидатов на должность менеджера

In [22]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_mupta_ru', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
5,speaker_27_center_83.mov,0.761,0.711,0.654,0.828,0.507,73.456
1,speaker_01_center_83.mov,0.765,0.697,0.655,0.759,0.493,70.910
4,speaker_19_center_83.mov,0.767,0.658,0.654,0.801,0.464,70.716
9,speaker_23_center_83.mov,0.701,0.688,0.620,0.808,0.451,70.398
8,speaker_06_center_83.mov,0.687,0.659,0.612,0.750,0.421,67.147
7,speaker_24_center_83.mov,0.710,0.662,0.610,0.711,0.413,66.379
2,speaker_15_center_83.mov,0.671,0.671,0.602,0.708,0.399,65.828
3,speaker_07_center_83.mov,0.672,0.663,0.573,0.706,0.382,64.962
10,speaker_11_center_83.mov,0.718,0.599,0.573,0.732,0.380,64.213


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [23]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [24]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_mupta_ru', out = True)

# Опционно
df = _b5.df_files_priority_skill_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Analytical,Interactive,Routine,Non-Routine
Person ID,,,,,,,,,,
5,speaker_27_center_83.mov,0.761,0.711,0.654,0.828,0.507,0.445,0.473,0.655,0.530
9,speaker_23_center_83.mov,0.701,0.688,0.620,0.808,0.451,0.388,0.395,0.561,0.461
4,speaker_19_center_83.mov,0.767,0.658,0.654,0.801,0.464,0.383,0.390,0.560,0.459
1,speaker_01_center_83.mov,0.765,0.697,0.655,0.759,0.493,0.382,0.393,0.560,0.460
8,speaker_06_center_83.mov,0.687,0.659,0.612,0.750,0.421,0.366,0.374,0.533,0.437
7,speaker_24_center_83.mov,0.710,0.662,0.610,0.711,0.413,0.357,0.369,0.525,0.430
2,speaker_15_center_83.mov,0.671,0.671,0.602,0.708,0.399,0.357,0.368,0.522,0.427
3,speaker_07_center_83.mov,0.672,0.663,0.573,0.706,0.382,0.353,0.365,0.516,0.424
10,speaker_11_center_83.mov,0.718,0.599,0.573,0.732,0.380,0.347,0.356,0.510,0.418


<hr>

Для ранжирования кандидатов по одному из шестнадцати типов личности MBTI необходимо задать матрицу корреляции между персональными качествами личности человека и четырьмя диспозициями MBTI, установить порог полярности качеств и указать целевой тип личности MBTI.

В качестве примера предлагается использование коэффициентов корреляции, представленных в статье [1]. Описание типов личности MBTI и соотвествующие им успешные профессии представлены в статье [2].

1) Furnham A. The big five facets and the MBTI: The relationship between the 30 NEO-PI (R) Facets and the four Myers-Briggs Type Indicator (MBTI) scores // Psychology. - 2022. vol. 13(10). - pp. 1504-1516.
2) Tieger P.D., Barron B., Tieger K. Do what you are: Discover the perfect career for you through the secrets of personality type // Hachette UK. - 2024.

##### Типы личности MBTI основаны на четырех измерениях личности:

<div style="text-align: center;">
  <table style="width:100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Описание измерения</th>
      <th>Измерение</th>
    </tr>
    <tr>
      <td>Как мы взаимодействуем с миром и куда направляем свою энергию</td>
      <td>(E) Экстраверсия - Интроверсия (I)</td>
    </tr>
    <tr>
      <td>Вид информации, которую мы естественным образом замечаем</td>
      <td>(S) Сенсорика - Интуиция (N)</td>
    </tr>
    <tr>
      <td>Как мы принимаем решения</td>
      <td>(T) Логика - Чувства (F)</td>
    </tr>
    <tr>
      <td>Предпочитаем ли мы жить более структурированно (принимая решения) или более спонтанно (принимая информацию)</td>
      <td>(J) Оценка - Восприятие (P)</td>
    </tr>
  </table>
</div>

##### Типы личности MBTI и соотвествующие им успешные профессии:

<div style="text-align: center;">
  <table style="width: 100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Тип личности</th>
      <th>Описание</th>
      <th>Успешные профессии</th>
    </tr>
    <tr>
      <td>ISTJ</td>
      <td>Этот человек отличается ответственностью, строгостью и педантичностью. Он опирается на объективные факты и склонен к аналитическому мышлению. Приступает к задаче только тогда, когда уверен в своих возможностях и успехе</td>
      <td>Инспектор: бухгалтер, аудитор, бюджетный аналитик, финансовый менеджер, разработчик, системный аналитик, библиотекарь и т. д.</td>
    </tr>
    <tr>
      <td>ISFJ</td>
      <td>Этот человек склонен к самоанализу и анализу окружающих, легко распознает фальшь и предпочитает сохранять психологическую дистанцию. Он исполнителен, внимателен и готов помогать другим. Его силы и энергия исходят из внутренних ресурсов, и он всегда полагается на собственный опыт</td>
      <td>Защитник: медсестра, врач, ветеринар или ветеринарный ассистент, социальный работник, сельскохозяйственный или пищевой ученый, секретарь, водитель и т. д.</td>
    </tr>
    <tr>
      <td>INFJ</td>
      <td>О таких людях говорят: «ему можно доверять». Он отличается высокой чувствительностью, уделяет большое внимание межличностным отношениям, умеет давать ценные советы и помогает раскрывать потенциал других. Развитая интуиция не только генерирует множество идей, но и способствует самоорганизации</td>
      <td>Советник: психолог, специалист по управлению персоналом, офис-менеджер, специалист по обучению, графический дизайнер и т. д.</td>
    </tr>
    <tr>
      <td>INTJ</td>
      <td>Этот человек умеет выделять главное, говорит четко и по существу, придерживается практического подхода. Он стремится постоянно улучшать свою работу и всегда ищет способы сделать задачу еще лучше. Пустые разговоры ему не по душе, поэтому он избегает больших шумных компаний и с трудом заводит новые знакомства</td>
      <td>Мастермайнд: аниматор, архитектор, копирайтер, фотограф, тележурналист, видеомонтажер, специалист по бизнес-развитию, исполнительный директор, профессор и т. д.</td>
    </tr>
    <tr>
      <td>ISTP</td>
      <td>Этот человек воспринимает мир через ощущения. По природе эмпат, но чаще сосредоточен на себе. Его умение объективно принимать решения и анализировать ситуацию указывает на технический склад ума. Он всегда соблюдает дедлайны, хотя иногда может поступить неожиданно</td>
      <td>Создатель: инженер, техник, строитель, инспектор, судебный эксперт, программист, разработчик ПО и т. д.</td>
    </tr>
    <tr>
      <td>ISFP</td>
      <td>Этот человек умеет находить радость в однообразии и рутинных делах. Прекрасно ладит с людьми, избегая конфликтов. Ему важно чувствовать свою значимость и оказывать помощь. Такой человек не стремится руководить или менять других, уважает их личные границы и ожидает того же в ответ. По натуре он приземленный практик, на которого всегда можно положиться</td>
      <td>Композитор: помощник по маркетингу, танцор, шеф-повар, офис-администратор, художник, дизайнер интерьеров, секретарь, медсестра и т. д.</td>
    </tr>
    <tr>
      <td>INFP</td>
      <td>Этот человек - чувствительный лирик, прекрасно разбирающийся в людях и легко вызывающий у них симпатию. Он обладает отличным чувством юмора и уделяет большое внимание своему внешнему виду. Стремится к самопознанию, гармонии с собой и старается быть полезным окружающим</td>
      <td>Целитель: писатель, дизайнер мультимедиа, менеджер по работе с клиентами, учитель для детей с особыми потребностями, тренер, редактор, модельер и т. д.</td>
    </tr>
    <tr>
      <td>INTP</td>
      <td>Этот человек - эрудит с философским складом ума. Он тщательно анализирует свои решения, стремясь к объективности и беспристрастности. Бурные проявления эмоций ему не свойственны. Однако большое количество данных и их изменчивость могут вызывать у него внутреннее напряжение</td>
      <td>Архитектор: технический писатель, веб-разработчик, аналитик информационной безопасности, исследователь, ученый, юрист и т. д.</td>
    </tr>
    <tr>
      <td>ESTP</td>
      <td>Этот человек всегда добивается успеха, невзирая на препятствия, которые лишь усиливают его целеустремленность. Он стремится к лидерским позициям и плохо переносит роль подчиненного. Обычно разрабатывает четкий план действий и неуклонно ему следует</td>
      <td>Промоутер: специалист по работе с клиентами, актер, личный тренер, бренд-амбассадор, менеджер, предприниматель, креативный директор, полицейский, маркетолог, производитель и т. д.</td>
    </tr>
    <tr>
      <td>ESFP</td>
      <td>Этот человек легко выявляет слабые стороны людей, что позволяет ему эффективно манипулировать и управлять. В общении он чаще всего руководствуется собственными интересами и предпочитает жить в настоящем. Часто не завершает начатое, стремясь к быстрым результатам. Однако при этом стремится поддерживать гармоничные отношения с окружающими</td>
      <td>Исполнитель: бортпроводник, артист, учитель, менеджер по связям с общественностью, торговый представитель, организатор мероприятий и т. д.</td>
    </tr>
    <tr>
      <td>ENFP</td>
      <td>Этот человек - творческая личность и фантазер, обладающий качествами, которые помогают ему успешно взаимодействовать с другими, быть открытым и общительным. Он активно участвует в различных мероприятиях, легко решает возникающие вопросы и демонстрирует гибкость</td>
      <td>Чемпион: медицинский работник, продюсер, продавец-консультант, специалист по обслуживанию клиентов, сценарист, ведущий на ТВ/радио и т. д.</td>
    </tr>
    <tr>
      <td>ENTP</td>
      <td>Этот человек - изобретательный, инициативный и гибкий. Он генератор идей и первопроходец, который не выносит рутины. Постоянное движение и интуитивное принятие решений всегда сопровождают его в работе</td>
      <td>Новатор: инженер, маркетолог, менеджер по социальным сетям, аналитик управления, руководитель цифрового маркетинга, бизнес-консультант, разработчик игр, менеджер по продажам и т. д.</td>
    </tr>
    <tr>
      <td>ESTJ</td>
      <td>Это трудолюбивый человек, который воспринимает мир таким, какой он есть. Он склонен тщательно планировать и доводить дела до конца. Заботится о своем ближайшем окружении, проявляет добродушие, но иногда может быть вспыльчивым, резким и упрямым</td>
      <td>Супервайзер: управляющий директор, менеджер отеля, финансовый сотрудник, судья, агент по недвижимости, генеральный директор, шеф-повар, менеджер по бизнес-развитию, телемаркетолог и т. д.</td>
    </tr>
    <tr>
      <td>ESFJ</td>
      <td>Этот человек умеет оказывать влияние на людей, проявляет заботу и готов жертвовать собой ради других. Он легко устанавливает контакт с любым человеком и способен направить ситуацию в нужное ему русло</td>
      <td>Поставщик: специалист по технической поддержке, менеджер по работе с клиентами, профессор колледжа, медицинский исследователь, бухгалтер, фотожурналист и т. д.</td>
    </tr>
    <tr>
      <td>ENFJ</td>
      <td>Этот человек отличается эмоциональностью и эмпатией. Его мимика выразительна, а речь — красноречива. Благодаря своей самоорганизованности, он успешно воплощает свои фантазии и идеи в жизнь. Он интуитивно понимает, какое решение следует принять в каждой конкретной ситуации</td>
      <td>Учитель: менеджер по связям с общественностью, менеджер по продажам, директор по управлению персоналом, арт-директор, консультант и т. д.</td>
    </tr>
    <tr>
      <td>ENTJ</td>
      <td>Этот человек легко увлекается, готов рисковать и полагается на интуицию. Без страха внедряет новые технологии и способен глубоко анализировать как себя, так и окружающий мир. Жизнь для него - это борьба, в которой он чувствует себя уверенно. Открыт для новых возможностей, но при этом нуждается в контроле</td>
      <td>Командир: руководитель строительства, администратор службы здравоохранения, финансовый бухгалтер, аудитор, юрист, директор школы, химический инженер, менеджер баз данных и т. д.</td>
    </tr>
  </table>
</div>

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим типам личности.

In [25]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/493644095?token=EX7hFxNJhMoLumI&filename=df_mbti_correlation.csv'
df_correlation_coefficients = pd.read_csv(url)

df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,EI,SN,TF,JP
ID,,,,,
1,Openness,0.09,-0.03,-0.14,-0.16
2,Conscientiousness,0.04,-0.04,0.20,0.14
3,Extraversion,0.20,-0.03,0.01,-0.07
4,Agreeableness,0.02,0.05,-0.35,0.03
5,Non-Neuroticism,0.08,0.00,0.16,0.00


In [26]:
_b5._professional_match(
    correlation_coefficients = df_correlation_coefficients,
    personality_type = "ENFJ",
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5._df_files_MBTI_job_match, name = 'MBTI_ranking_mupta_ru', out = True)

# Опционно
df = _b5.df_files_MBTI_job_match_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:6]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,EI,SN,TF,JP,MBTI,MBTI_Score,Match
Person ID,,,,,,,,,,,,,
4,speaker_19_center_83.mov,0.767,0.658,0.654,0.801,0.464,0.205048,-0.028893,-0.323879,-0.052312,ENFP,0.418365,75.0
1,speaker_01_center_83.mov,0.765,0.697,0.655,0.759,0.493,0.203478,-0.032546,-0.305707,-0.048042,ENFP,0.406299,75.0
9,speaker_23_center_83.mov,0.701,0.688,0.620,0.808,0.451,0.194701,-0.026765,-0.309276,-0.035002,ENFP,0.398057,75.0
8,speaker_06_center_83.mov,0.687,0.659,0.612,0.750,0.421,0.191873,-0.027835,-0.287880,-0.037895,ENFP,0.380691,75.0
10,speaker_11_center_83.mov,0.718,0.599,0.573,0.732,0.380,0.187559,-0.026121,-0.292044,-0.049246,ENFP,0.379293,75.0
7,speaker_24_center_83.mov,0.710,0.662,0.610,0.711,0.413,0.193622,-0.030556,-0.275847,-0.042330,ENFP,0.375019,75.0
2,speaker_15_center_83.mov,0.671,0.671,0.602,0.708,0.399,0.189892,-0.029622,-0.265515,-0.034329,ENFP,0.363772,75.0
5,speaker_27_center_83.mov,0.761,0.711,0.654,0.828,0.507,0.284760,-0.029477,-0.166449,-0.043191,ENFP,0.360515,75.0
6,speaker_10_center_83.mov,0.698,0.599,0.572,0.675,0.351,0.186638,-0.028337,-0.264573,-0.047641,ENFP,0.359661,75.0


### `MuPTA` (en)

In [27]:
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

corpus = 'fi'
lang = 'en'

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url, force_reload = False)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang=lang)
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['fe']['googledisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url, force_reload = False)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['googledisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['text'][corpus]['nn']['googledisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url, force_reload = False)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['googledisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url, force_reload = False)

PATH_TO_DIR = './video_MuPTA/'
PATH_SAVE_VIDEO = './video_MuPTA/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса MuPTA
# URL: https://hci.nw.ru/en/pages/mupta-corpus
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '477995979?token=2cvyk7CS0mHx2MJ&filename=speaker_06_center_83.mov',
    '477995980?token=jGPtBPS69uzFU6Y&filename=speaker_01_center_83.mov',
    '477995967?token=zCaRbNB6ht5wMPq&filename=speaker_11_center_83.mov',
    '477995966?token=B1rbinDYRQKrI3T&filename=speaker_15_center_83.mov',
    '477995978?token=dEpVDtZg1EQiEQ9&filename=speaker_07_center_83.mov',
    '477995961?token=o1hVjw8G45q9L9Z&filename=speaker_19_center_83.mov',
    '477995964?token=5K220Aqf673VHPq&filename=speaker_23_center_83.mov',
    '477995965?token=v1LVD2KT1cU7Lpb&filename=speaker_24_center_83.mov',
    '477995962?token=tmaSGyyWLA6XCy9&filename=speaker_27_center_83.mov',
    '477995963?token=bTpo96qNDPcwGqb&filename=speaker_10_center_83.mov',
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mov'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['mupta']['googledisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = lang)

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:26:21</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:26:22</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... kaggle/working/OCEANAI/video_MuPTA/test/speaker_11_center_83.mov ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
Person ID,,,,,,
1,speaker_01_center_83.mov,0.590114,0.540202,0.437401,0.5878,0.510256
2,speaker_15_center_83.mov,0.580967,0.560311,0.503614,0.600134,0.519449
3,speaker_07_center_83.mov,0.440438,0.465158,0.287292,0.425223,0.395389
4,speaker_19_center_83.mov,0.509879,0.446402,0.42639,0.451874,0.447009
5,speaker_27_center_83.mov,0.560265,0.665581,0.425382,0.58864,0.566939
6,speaker_10_center_83.mov,0.477925,0.502658,0.373761,0.441541,0.425135
7,speaker_24_center_83.mov,0.426992,0.510604,0.299171,0.432961,0.442596
8,speaker_06_center_83.mov,0.661469,0.675205,0.604212,0.646378,0.643688
9,speaker_23_center_83.mov,0.502359,0.544019,0.309871,0.445746,0.455199


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:26:22</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.1643,0.1621,0.1768,0.2582,0.1099,0.1743
Accuracy,0.8357,0.8379,0.8232,0.7418,0.8901,0.8257


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-25 19:26:22</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.1743</span>, средняя точность: <span style="color:#1776D2">0.8257</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 556.537 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [28]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [29]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_mupta_en', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
8,speaker_06_center_83.mov,0.661,0.675,0.604,0.646,0.644,65.574
5,speaker_27_center_83.mov,0.560,0.666,0.425,0.589,0.567,57.458
2,speaker_15_center_83.mov,0.581,0.560,0.504,0.600,0.519,56.765
1,speaker_01_center_83.mov,0.590,0.540,0.437,0.588,0.510,56.267
9,speaker_23_center_83.mov,0.502,0.544,0.310,0.446,0.455,48.342
4,speaker_19_center_83.mov,0.510,0.446,0.426,0.452,0.447,47.805
6,speaker_10_center_83.mov,0.478,0.503,0.374,0.442,0.425,46.305
7,speaker_24_center_83.mov,0.427,0.511,0.299,0.433,0.443,43.638
3,speaker_07_center_83.mov,0.440,0.465,0.287,0.425,0.395,42.745


#### Ранжирование кандидатов на должность менеджера

In [30]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_mupta_en', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
8,speaker_06_center_83.mov,0.661,0.675,0.604,0.646,0.644,65.227
5,speaker_27_center_83.mov,0.560,0.666,0.425,0.589,0.567,58.574
2,speaker_15_center_83.mov,0.581,0.560,0.504,0.600,0.519,56.481
1,speaker_01_center_83.mov,0.590,0.540,0.437,0.588,0.510,54.505
9,speaker_23_center_83.mov,0.502,0.544,0.310,0.446,0.455,46.873
6,speaker_10_center_83.mov,0.478,0.503,0.374,0.442,0.425,45.740
4,speaker_19_center_83.mov,0.510,0.446,0.426,0.452,0.447,45.459
7,speaker_24_center_83.mov,0.427,0.511,0.299,0.433,0.443,43.965
3,speaker_07_center_83.mov,0.440,0.465,0.287,0.425,0.395,41.930


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [31]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [32]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_mupta_en', out = True)

# Опционно
df = _b5.df_files_priority_skill_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Analytical,Interactive,Routine,Non-Routine
Person ID,,,,,,,,,,
8,speaker_06_center_83.mov,0.661,0.675,0.604,0.646,0.644,0.408,0.443,0.604,0.487
5,speaker_27_center_83.mov,0.560,0.666,0.425,0.589,0.567,0.375,0.430,0.540,0.464
2,speaker_15_center_83.mov,0.581,0.560,0.504,0.600,0.519,0.353,0.380,0.520,0.421
1,speaker_01_center_83.mov,0.590,0.540,0.437,0.588,0.510,0.343,0.390,0.496,0.427
6,speaker_10_center_83.mov,0.478,0.503,0.374,0.442,0.425,0.170,0.188,0.206,0.205
7,speaker_24_center_83.mov,0.427,0.511,0.299,0.433,0.443,0.167,0.187,0.205,0.202
9,speaker_23_center_83.mov,0.502,0.544,0.310,0.446,0.455,0.167,0.219,0.261,0.245
3,speaker_07_center_83.mov,0.440,0.465,0.287,0.425,0.395,0.086,0.067,0.067,0.096
4,speaker_19_center_83.mov,0.510,0.446,0.426,0.452,0.447,0.084,0.094,0.118,0.137


<hr>

Для ранжирования кандидатов по одному из шестнадцати типов личности MBTI необходимо задать матрицу корреляции между персональными качествами личности человека и четырьмя диспозициями MBTI, установить порог полярности качеств и указать целевой тип личности MBTI.

В качестве примера предлагается использование коэффициентов корреляции, представленных в статье [1]. Описание типов личности MBTI и соотвествующие им успешные профессии представлены в статье [2].

1) Furnham A. The big five facets and the MBTI: The relationship between the 30 NEO-PI (R) Facets and the four Myers-Briggs Type Indicator (MBTI) scores // Psychology. - 2022. vol. 13(10). - pp. 1504-1516.
2) Tieger P.D., Barron B., Tieger K. Do what you are: Discover the perfect career for you through the secrets of personality type // Hachette UK. - 2024.

##### Типы личности MBTI основаны на четырех измерениях личности:

<div style="text-align: center;">
  <table style="width:100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Описание измерения</th>
      <th>Измерение</th>
    </tr>
    <tr>
      <td>Как мы взаимодействуем с миром и куда направляем свою энергию</td>
      <td>(E) Экстраверсия - Интроверсия (I)</td>
    </tr>
    <tr>
      <td>Вид информации, которую мы естественным образом замечаем</td>
      <td>(S) Сенсорика - Интуиция (N)</td>
    </tr>
    <tr>
      <td>Как мы принимаем решения</td>
      <td>(T) Логика - Чувства (F)</td>
    </tr>
    <tr>
      <td>Предпочитаем ли мы жить более структурированно (принимая решения) или более спонтанно (принимая информацию)</td>
      <td>(J) Оценка - Восприятие (P)</td>
    </tr>
  </table>
</div>

##### Типы личности MBTI и соотвествующие им успешные профессии:

<div style="text-align: center;">
  <table style="width: 100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Тип личности</th>
      <th>Описание</th>
      <th>Успешные профессии</th>
    </tr>
    <tr>
      <td>ISTJ</td>
      <td>Этот человек отличается ответственностью, строгостью и педантичностью. Он опирается на объективные факты и склонен к аналитическому мышлению. Приступает к задаче только тогда, когда уверен в своих возможностях и успехе</td>
      <td>Инспектор: бухгалтер, аудитор, бюджетный аналитик, финансовый менеджер, разработчик, системный аналитик, библиотекарь и т. д.</td>
    </tr>
    <tr>
      <td>ISFJ</td>
      <td>Этот человек склонен к самоанализу и анализу окружающих, легко распознает фальшь и предпочитает сохранять психологическую дистанцию. Он исполнителен, внимателен и готов помогать другим. Его силы и энергия исходят из внутренних ресурсов, и он всегда полагается на собственный опыт</td>
      <td>Защитник: медсестра, врач, ветеринар или ветеринарный ассистент, социальный работник, сельскохозяйственный или пищевой ученый, секретарь, водитель и т. д.</td>
    </tr>
    <tr>
      <td>INFJ</td>
      <td>О таких людях говорят: «ему можно доверять». Он отличается высокой чувствительностью, уделяет большое внимание межличностным отношениям, умеет давать ценные советы и помогает раскрывать потенциал других. Развитая интуиция не только генерирует множество идей, но и способствует самоорганизации</td>
      <td>Советник: психолог, специалист по управлению персоналом, офис-менеджер, специалист по обучению, графический дизайнер и т. д.</td>
    </tr>
    <tr>
      <td>INTJ</td>
      <td>Этот человек умеет выделять главное, говорит четко и по существу, придерживается практического подхода. Он стремится постоянно улучшать свою работу и всегда ищет способы сделать задачу еще лучше. Пустые разговоры ему не по душе, поэтому он избегает больших шумных компаний и с трудом заводит новые знакомства</td>
      <td>Мастермайнд: аниматор, архитектор, копирайтер, фотограф, тележурналист, видеомонтажер, специалист по бизнес-развитию, исполнительный директор, профессор и т. д.</td>
    </tr>
    <tr>
      <td>ISTP</td>
      <td>Этот человек воспринимает мир через ощущения. По природе эмпат, но чаще сосредоточен на себе. Его умение объективно принимать решения и анализировать ситуацию указывает на технический склад ума. Он всегда соблюдает дедлайны, хотя иногда может поступить неожиданно</td>
      <td>Создатель: инженер, техник, строитель, инспектор, судебный эксперт, программист, разработчик ПО и т. д.</td>
    </tr>
    <tr>
      <td>ISFP</td>
      <td>Этот человек умеет находить радость в однообразии и рутинных делах. Прекрасно ладит с людьми, избегая конфликтов. Ему важно чувствовать свою значимость и оказывать помощь. Такой человек не стремится руководить или менять других, уважает их личные границы и ожидает того же в ответ. По натуре он приземленный практик, на которого всегда можно положиться</td>
      <td>Композитор: помощник по маркетингу, танцор, шеф-повар, офис-администратор, художник, дизайнер интерьеров, секретарь, медсестра и т. д.</td>
    </tr>
    <tr>
      <td>INFP</td>
      <td>Этот человек - чувствительный лирик, прекрасно разбирающийся в людях и легко вызывающий у них симпатию. Он обладает отличным чувством юмора и уделяет большое внимание своему внешнему виду. Стремится к самопознанию, гармонии с собой и старается быть полезным окружающим</td>
      <td>Целитель: писатель, дизайнер мультимедиа, менеджер по работе с клиентами, учитель для детей с особыми потребностями, тренер, редактор, модельер и т. д.</td>
    </tr>
    <tr>
      <td>INTP</td>
      <td>Этот человек - эрудит с философским складом ума. Он тщательно анализирует свои решения, стремясь к объективности и беспристрастности. Бурные проявления эмоций ему не свойственны. Однако большое количество данных и их изменчивость могут вызывать у него внутреннее напряжение</td>
      <td>Архитектор: технический писатель, веб-разработчик, аналитик информационной безопасности, исследователь, ученый, юрист и т. д.</td>
    </tr>
    <tr>
      <td>ESTP</td>
      <td>Этот человек всегда добивается успеха, невзирая на препятствия, которые лишь усиливают его целеустремленность. Он стремится к лидерским позициям и плохо переносит роль подчиненного. Обычно разрабатывает четкий план действий и неуклонно ему следует</td>
      <td>Промоутер: специалист по работе с клиентами, актер, личный тренер, бренд-амбассадор, менеджер, предприниматель, креативный директор, полицейский, маркетолог, производитель и т. д.</td>
    </tr>
    <tr>
      <td>ESFP</td>
      <td>Этот человек легко выявляет слабые стороны людей, что позволяет ему эффективно манипулировать и управлять. В общении он чаще всего руководствуется собственными интересами и предпочитает жить в настоящем. Часто не завершает начатое, стремясь к быстрым результатам. Однако при этом стремится поддерживать гармоничные отношения с окружающими</td>
      <td>Исполнитель: бортпроводник, артист, учитель, менеджер по связям с общественностью, торговый представитель, организатор мероприятий и т. д.</td>
    </tr>
    <tr>
      <td>ENFP</td>
      <td>Этот человек - творческая личность и фантазер, обладающий качествами, которые помогают ему успешно взаимодействовать с другими, быть открытым и общительным. Он активно участвует в различных мероприятиях, легко решает возникающие вопросы и демонстрирует гибкость</td>
      <td>Чемпион: медицинский работник, продюсер, продавец-консультант, специалист по обслуживанию клиентов, сценарист, ведущий на ТВ/радио и т. д.</td>
    </tr>
    <tr>
      <td>ENTP</td>
      <td>Этот человек - изобретательный, инициативный и гибкий. Он генератор идей и первопроходец, который не выносит рутины. Постоянное движение и интуитивное принятие решений всегда сопровождают его в работе</td>
      <td>Новатор: инженер, маркетолог, менеджер по социальным сетям, аналитик управления, руководитель цифрового маркетинга, бизнес-консультант, разработчик игр, менеджер по продажам и т. д.</td>
    </tr>
    <tr>
      <td>ESTJ</td>
      <td>Это трудолюбивый человек, который воспринимает мир таким, какой он есть. Он склонен тщательно планировать и доводить дела до конца. Заботится о своем ближайшем окружении, проявляет добродушие, но иногда может быть вспыльчивым, резким и упрямым</td>
      <td>Супервайзер: управляющий директор, менеджер отеля, финансовый сотрудник, судья, агент по недвижимости, генеральный директор, шеф-повар, менеджер по бизнес-развитию, телемаркетолог и т. д.</td>
    </tr>
    <tr>
      <td>ESFJ</td>
      <td>Этот человек умеет оказывать влияние на людей, проявляет заботу и готов жертвовать собой ради других. Он легко устанавливает контакт с любым человеком и способен направить ситуацию в нужное ему русло</td>
      <td>Поставщик: специалист по технической поддержке, менеджер по работе с клиентами, профессор колледжа, медицинский исследователь, бухгалтер, фотожурналист и т. д.</td>
    </tr>
    <tr>
      <td>ENFJ</td>
      <td>Этот человек отличается эмоциональностью и эмпатией. Его мимика выразительна, а речь — красноречива. Благодаря своей самоорганизованности, он успешно воплощает свои фантазии и идеи в жизнь. Он интуитивно понимает, какое решение следует принять в каждой конкретной ситуации</td>
      <td>Учитель: менеджер по связям с общественностью, менеджер по продажам, директор по управлению персоналом, арт-директор, консультант и т. д.</td>
    </tr>
    <tr>
      <td>ENTJ</td>
      <td>Этот человек легко увлекается, готов рисковать и полагается на интуицию. Без страха внедряет новые технологии и способен глубоко анализировать как себя, так и окружающий мир. Жизнь для него - это борьба, в которой он чувствует себя уверенно. Открыт для новых возможностей, но при этом нуждается в контроле</td>
      <td>Командир: руководитель строительства, администратор службы здравоохранения, финансовый бухгалтер, аудитор, юрист, директор школы, химический инженер, менеджер баз данных и т. д.</td>
    </tr>
  </table>
</div>

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим типам личности.

In [33]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/493644095?token=EX7hFxNJhMoLumI&filename=df_mbti_correlation.csv'
df_correlation_coefficients = pd.read_csv(url)

df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,EI,SN,TF,JP
ID,,,,,
1,Openness,0.09,-0.03,-0.14,-0.16
2,Conscientiousness,0.04,-0.04,0.20,0.14
3,Extraversion,0.20,-0.03,0.01,-0.07
4,Agreeableness,0.02,0.05,-0.35,0.03
5,Non-Neuroticism,0.08,0.00,0.16,0.00


In [34]:
_b5._professional_match(
    correlation_coefficients = df_correlation_coefficients,
    personality_type = "ENFJ",
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5._df_files_MBTI_job_match, name = 'MBTI_ranking_mupta_en', out = True)

# Опционно
df = _b5.df_files_MBTI_job_match_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:6]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,EI,SN,TF,JP,MBTI,MBTI_Score,Match
Person ID,,,,,,,,,,,,,
8,speaker_06_center_83.mov,0.661,0.675,0.604,0.646,0.644,0.271805,-0.032660,-0.074765,-0.034210,ENFP,0.284422,75.0
2,speaker_15_center_83.mov,0.581,0.560,0.504,0.600,0.519,0.228981,-0.024943,-0.091172,-0.031760,ENFP,0.258822,75.0
5,speaker_27_center_83.mov,0.560,0.666,0.425,0.589,0.567,0.049099,-0.001238,-0.064888,0.050975,ENFJ,0.166200,100.0
1,speaker_01_center_83.mov,0.590,0.540,0.437,0.588,0.510,0.039815,0.003201,-0.103039,0.029462,ESFJ,0.129236,75.0
6,speaker_10_center_83.mov,0.478,0.503,0.374,0.442,0.425,-0.140501,-0.016633,0.250221,0.159757,INTJ,0.088195,50.0
7,speaker_24_center_83.mov,0.427,0.511,0.299,0.433,0.443,-0.121906,-0.020287,0.239629,0.147756,INTJ,0.084022,50.0
4,speaker_19_center_83.mov,0.510,0.446,0.426,0.452,0.447,-0.102043,-0.007242,-0.078293,-0.127786,INFP,0.042768,50.0
9,speaker_23_center_83.mov,0.502,0.544,0.310,0.446,0.455,-0.040332,-0.049823,0.118554,0.004104,INTJ,0.026963,50.0
10,speaker_11_center_83.mov,0.403,0.344,0.317,0.421,0.383,-0.152416,0.014283,0.070540,0.025814,ISTJ,0.006454,25.0
